In [3]:
!pip install selenium

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached selenium-4.1.5-py3-none-any.whl (979 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached trio-0.20.0-py3-none-any.whl (359 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached outcome-1.1.0-py2.py3-none-any.whl (9.7 kB)
  Using cached attrs-21.4.0-py2.py3-none-any.whl (60 kB)
  Using cached sniffio-1.2.0-py3-none-any.whl (10 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.1.0-py3-none-any.whl (24 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached cryptography-37.0.2-cp36-abi3-macosx_10_10_universal2.whl (5.3 MB)
  Using cached pyOpenSSL-22.0.0-py2.py3-none-any.whl (55 kB)
  Using cached cffi-1

In [4]:
import selenium
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [5]:
def prepare_driver(url):
    '''Returns a Firefox Webdriver.'''
    options = Options()
    options.add_argument('-headless')
    driver = Firefox(executable_path='/opt/homebrew/bin/geckodriver', options=options)
    driver.get(url)
    wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.ID, 'ss')))
    return driver
 

In [6]:
def fill_form(driver, search_argument):
    '''Receives a search_argument to insert it in the search bar and 
    then clicks the search button.'''
 
    search_field = driver.find_element_by_id('ss')
    search_field.send_keys(search_argument)
    # We look for the search button and click it
    driver.find_element_by_class_name('sb-searchbox__button')\
        .click()
    wait = WebDriverWait(driver, timeout=10).until(
        EC.presence_of_all_elements_located(
            (By.CLASS_NAME, 'sr-hotel__title')))

In [7]:
def scrape_results(driver, n_results):
    '''Returns the data from n_results amount of results.'''
 
    accommodations_urls = list()
    accommodations_data = list()
    
    # Get the accommodations links
    for accomodation_title in driver.find_elements_by_class_name('sr-hotel__title'):
        accommodations_urls.append(accomodation_title.find_element_by_class_name(
            'hotel_name_link').get_attribute('href'))
 
    # Iterate over a defined range and scrape the links
    for url in range(0, n_results):
        if url == n_results:
            break
        url_data = scrape_accommodation_data(driver, accommodations_urls[url])
        accommodations_data.append(url_data)
    
    return accommodations_data
 

In [8]:
accommodation_fields = dict()

In [9]:
driver.get(accommodation_url)
time.sleep(10)
 

NameError: name 'driver' is not defined

In [11]:
import selenium
import json
import time
import re
import string
import requests
import bs4
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
 
domain =  'https://www.booking.com'
 
def prepare_driver(url):
    '''Returns a Firefox Webdriver.'''
    options = Options()
    # options.add_argument('-headless')
    driver = Firefox(executable_path='/opt/homebrew/bin/geckodriver', options=options)
    driver.get(url)
    wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.ID, 'ss')))
    return driver
 
def fill_form(driver, search_argument):
    '''Finds all the input tags in form and makes a POST requests.'''
    search_field = driver.find_element_by_id('ss')
    search_field.send_keys(search_argument)
    # We look for the search button and click it
    driver.find_element_by_class_name('sb-searchbox__button')\
        .click()
    wait = WebDriverWait(driver, timeout=10).until(
        EC.presence_of_all_elements_located(
            (By.CLASS_NAME, 'sr-hotel__title')))
 
def scrape_results(driver, n_results):
    '''Returns the data from n_results amount of results.'''
 
    accommodations_urls = list()
    accommodations_data = list()
 
    for accomodation_title in driver.find_elements_by_class_name('sr-hotel__title'):
        accommodations_urls.append(accomodation_title.find_element_by_class_name(
            'hotel_name_link').get_attribute('href'))
 
    for url in range(0, n_results):
        if url == n_results:
            break
        url_data = scrape_accommodation_data(driver, accommodations_urls[url])
        accommodations_data.append(url_data)
    
    return accommodations_data
 
def scrape_accommodation_data(driver, accommodation_url):
    '''Visits an accommodation page and extracts the data.'''
 
    if driver == None:
        driver = prepare_driver(accommodation_url)
 
    driver.get(accommodation_url)
    time.sleep(12)
 
    accommodation_fields = dict()
 
    # Get the accommodation name
    accommodation_fields['name'] = driver.find_element_by_id('hp_hotel_name')\
        .text.strip('Hotel')
 
    # Get the accommodation score
    accommodation_fields['score'] = driver.find_element_by_class_name(
        'bui-review-score--end').find_element_by_class_name(
        'bui-review-score__badge').text
    
    # Get the accommodation location
    accommodation_fields['location'] = driver.find_element_by_id('showMap2')\
        .find_element_by_class_name('hp_address_subtitle').text
 
    # Get the most popular facilities
    accommodation_fields['popular_facilities'] = list()
    facilities = driver.find_element_by_class_name('hp_desc_important_facilities')
 
    for facility in facilities.find_elements_by_class_name('important_facility'):
        accommodation_fields['popular_facilities'].append(facility.text)
    
    return accommodation_fields
 
if __name__ == '__main__':
 
    try:
        driver = prepare_driver(domain)
        fill_form(driver, 'Barcelona')
        accommodations_data = scrape_results(driver, 10)
        accommodations_data = json.dumps(accommodations_data, indent=4)
        with open('booking_data.json', 'w') as f:
            f.write(accommodations_data)
    finally:
        driver.quit()


/var/folders/26/r_bcwcfx0x7_t9wv6jxbj2fc0000gp/T/ipykernel_24699/1386163876.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Firefox(executable_path='/opt/homebrew/bin/geckodriver', options=options)
/var/folders/26/r_bcwcfx0x7_t9wv6jxbj2fc0000gp/T/ipykernel_24699/1386163876.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_field = driver.find_element_by_id('ss')
/var/folders/26/r_bcwcfx0x7_t9wv6jxbj2fc0000gp/T/ipykernel_24699/1386163876.py:32: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('sb-searchbox__button')\


ElementClickInterceptedException: Message: Element <button class="sb-searchbox__button " type="submit"> is not clickable at point (1100,523) because another element <button id="onetrust-accept-btn-handler"> obscures it
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.jsm:282:5
webdriverClickElement@chrome://remote/content/marionette/interaction.js:166:11
interaction.clickElement@chrome://remote/content/marionette/interaction.js:125:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:203:24
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:91:31
